In [5]:
using JuMP
import LinearAlgebra    
import SCS

In [6]:
N, d = 2, 2

(2, 2)

In [7]:
function random_state(d)
    x = randn(ComplexF64, (d, d))
    y = x * x'
    return y
    #return LinearAlgebra.Hermitian(y / LinearAlgebra.tr(y))
end

#ρ = [random_state(d) for i in 1:N][1]
#ρ = 0.5 * [LinearAlgebra.Hermitian([[1,0] [0,1]])]
#ρ = 0.5 * [[[1,1] [1,1]],[[1,-1] [-1,1]]] # discriminate between plus and minus
ρ = [[[1,0] [0,0]],0.5 *[[1,1] [1,1]]] # discrimate between 0 and plus



2-element Vector{Matrix{Float64}}:
 [1.0 0.0; 0.0 0.0]
 [0.5 0.5; 0.5 0.5]

In [8]:
model = Model(SCS.Optimizer)
set_silent(model)

In [9]:
E = [@variable(model, [1:d, 1:d] in HermitianPSDCone()) for i in 1:N]

2-element Vector{LinearAlgebra.Hermitian{GenericAffExpr{ComplexF64, VariableRef}, Matrix{GenericAffExpr{ComplexF64, VariableRef}}}}:
 [_[1] _[2] + _[4] im; _[2] - _[4] im _[3]]
 [_[5] _[6] + _[8] im; _[6] - _[8] im _[7]]

In [10]:
@constraint(model, sum(E) == LinearAlgebra.I)

[_[1] + _[5] - 1                  _[2] + _[6] + _[4] im + _[8] im
 _[2] + _[6] - _[4] im - _[8] im  _[3] + _[7] - 1] ∈ Zeros()

In [11]:
@objective(
    model,
    Max,
    sum(real(LinearAlgebra.tr(ρ[i] * E[i])) for i in 1:N) / N,
)

0.5 _[1] + 0.25 _[5] + 0.5 _[6] + 0.25 _[7]

In [12]:
optimize!(model)
assert_is_solved_and_feasible(model)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 8.53552e-01
  Dual objective value : 8.53552e-01

* Work counters
  Solve time (sec)   : 1.28734e-02


In [13]:
objective_value(model)

0.8535516271731516

In [14]:
0.5 + 0.25 * sum(LinearAlgebra.svdvals(ρ[1] - ρ[2]))

0.8535533905932737

In [15]:
solution = [value.(e) for e in E]

2-element Vector{Matrix{ComplexF64}}:
 [0.8535516273557899 + 0.0im -0.35355163029489334 + 0.0im; -0.35355163029489334 + 0.0im 0.1464483663886332 + 0.0im]
 [0.1464483661466257 + 0.0im 0.3535516305211815 + 0.0im; 0.3535516305211815 - 0.0im 0.8535516267920372 + 0.0im]